In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from hyperopt import hp, tpe, Trials, STATUS_OK, fmin
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, PowerTransformer, QuantileTransformer
import category_encoders as ce
import warnings
warnings.filterwarnings("ignore")

In [ ]:
class morgan_fp:
    def __init__(self, radius, length):
        self.radius = radius
        self.length = length
    def __call__(self, smiles):
        mol = Chem.MolFromSmiles(smiles)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, self.radius, self.length)
        npfp = np.array(list(fp.ToBitString())).astype('float32')
        return npfp

In [ ]:
data_salt = pd.read_excel('final_data_041621_new.xlsx')
data_NaSO4 = data_salt[['Monomer A1 type', 'Monomer A2 type', 'a-mw', 'a-smile', 'b-mw',
       'b-smile', 'A1/A2 ratio', 'Monomer A concentration', 'Monomer B type',
       'Monomer B concentration', 'Organic solvent type',
       ' Additive X1 type in aqueous phase', 'Additive X1 concentration',
       ' Additive X2 type in aqueous phase', 'Additive X2 concentration',
       'Aqueous phase pH', ' Additive Y type in organic  phase',
       'Additive Y concentration', 'Nanomaterials type in aqueous phase',
       'Nanomaterials loading in aqueous phase',
       'Nanomaterials type in organic phase',
       'Nanomaterials loading in organnic phase', 'nanomaterials morphology',
       'Polymerization time', 'Post-treatment time', 'post-treatment T',
       'Substrate membrane type', 'substrate membrane pore size',
        'TMP',
        'Na+ valence', 'Na+ ionic radius', 'Na+ hydrated radius', 'Na+ stokes radius', 'Na+ hydration free energy',
        'SO42- valence', 'SO42-  ionic radius', 'SO42-  hydrated radius', 'SO42-  stokes radius', 'SO42-  hydration free energy',     
       'Na2SO4 concentration', 
       'Na2SO4 rejection']].copy()
data_NaSO4.dropna(subset=['Na2SO4 rejection'], axis =0, inplace=True)
data_NaSO4.reset_index(drop=True, inplace=True)

In [ ]:
train_data_per = pd.read_csv('per_train_data.csv')
train_data_salt = pd.read_csv('salt_train_data.csv')


In [ ]:
#scaler = StandardScaler()
def conv_data_pd_per(data, data_t, fp, en, scaler, numeric_features):
    data['a-fp'] = data['a-smile'].apply(fp)
    data['b-fp'] = data['b-smile'].apply(fp)
    x_a=np.array(list(data['a-fp']))
    x_b=np.array(list(data['b-fp']))
    for i in range(len(data)):
        if data['b-smile'][i]=='C':
            x_b[i]=x_b[i]*0
    
    X_train = data_t.drop(columns=['Monomer A1 type', 'a-smile','Monomer A2 type', 'b-smile', 'Permeability', 'a-fp', 'b-fp']).copy()
    Y_train = data_t['Permeability'].copy()
    hh=en.fit_transform(X_train, Y_train)
    SC= scaler.fit(hh[numeric_features])
    
    x = data.drop(columns=['Monomer A1 type', 'a-smile','Monomer A2 type', 'b-smile', 'Permeability', 'a-fp', 'b-fp']).copy()
    y = data['Permeability'].copy()
    xx = en.transform(x, y)
    col_pd = xx.drop(columns= numeric_features) #pd
    xxxx = SC.transform(xx[numeric_features])
    num_pd = pd.DataFrame(data= xxxx, columns=numeric_features) #pd
    
    xxxxx = np.concatenate((x_a, x_b), axis =1)
    fp_pd = pd.DataFrame(data= xxxxx, columns=[f'f_{i}' for i in range(2*x_a.shape[1])])
    
    x_pd = pd.concat([fp_pd, col_pd, num_pd], axis =1)
    y = data['Permeability'].values
    return x_pd, y

#scaler = StandardScaler()
def conv_data_pd_salt(data, data_t, fp, en, scaler, numeric_features):
    data['a-fp'] = data['a-smile'].apply(fp)
    data['b-fp'] = data['b-smile'].apply(fp)
    x_a=np.array(list(data['a-fp']))
    x_b=np.array(list(data['b-fp']))
    for i in range(len(data)):
        if data['b-smile'][i]=='C':
            x_b[i]=x_b[i]*0
    
    X_train = data_t.drop(columns=['Monomer A1 type', 'a-smile','Monomer A2 type', 'b-smile', 'salt rejection', 'a-fp', 'b-fp']).copy()
    Y_train = data_t['salt rejection'].copy()
    hh=en.fit_transform(X_train, Y_train)
    SC= scaler.fit(hh[numeric_features])
    
    x = data.drop(columns=['Monomer A1 type', 'a-smile','Monomer A2 type', 'b-smile', 'salt rejection',  'a-fp', 'b-fp']).copy()
    y = data['salt rejection'].copy()
    xx = en.transform(x, y)
    col_pd = xx.drop(columns= numeric_features) #pd
    xxxx = SC.transform(xx[numeric_features])
    num_pd = pd.DataFrame(data= xxxx, columns=numeric_features) #pd
    
    xxxxx = np.concatenate((x_a, x_b), axis =1)
    fp_pd = pd.DataFrame(data= xxxxx, columns=[f'f_{i}' for i in range(2*x_a.shape[1])])
    
    x_pd = pd.concat([fp_pd, col_pd, num_pd], axis =1)
    y = data['salt rejection'].values
    
    return x_pd, y

In [ ]:
categorical_features_per = train_data_per.select_dtypes(include=['object']).drop(['Monomer A1 type','a-smile', 'Monomer A2 type', 'b-smile'], axis=1).columns
numeric_features_per = train_data_per.select_dtypes(include=['int64', 'float64']).drop(['Permeability'], axis=1).columns

In [ ]:
import ast
result = pd.read_csv('per.csv')
result.sort_values('loss', ascending= True, inplace = True)
result.reset_index(drop = True, inplace =True)
params = ast.literal_eval(result['params'][0])
fp_per = morgan_fp(0, params['fp_length'])
sc_per = PowerTransformer()
en_per=ce.backward_difference.BackwardDifferenceEncoder(cols = categorical_features_per)
model_per = XGBRegressor(max_delta_step=params['max_delta_step'], learning_rate = params['learning_rate'],
                    max_depth = params['max_depth'], min_child_weight= params['min_child_weight'],
                    subsample=params['subsample'],reg_alpha=params['reg_alpha'],gamma= params['gamma'],
                    reg_lambda= params['reg_lambda'],n_estimators=params['n_estimators'], random_state=10)


In [ ]:
###Checking the model
model_per.load_model("per_model.model")
x_train_pd, y_train = conv_data_pd_per(train_data_per, train_data_per, fp_per, en_per, sc_per, numeric_features_per)
x_train=x_train_pd.values
r2_score(y_train, model_per.predict(x_train))

In [ ]:
categorical_features_salt = train_data_salt.select_dtypes(include=['object']).drop(['Monomer A1 type','a-smile', 'Monomer A2 type', 'b-smile'], axis=1).columns
numeric_features_salt = train_data_salt.select_dtypes(include=['int64', 'float64']).drop(['salt rejection'], axis=1).columns

In [ ]:
result = pd.read_csv('rej.csv')
result.sort_values('loss', ascending= True, inplace = True)
result.reset_index(drop = True, inplace =True)
params = ast.literal_eval(result['params'][0])
fp_salt = morgan_fp(params['fp_radius'], params['fp_length'])
sc_salt = RobustScaler()
en_salt =  ce.helmert.HelmertEncoder(cols = categorical_features_salt)
x_train_pd, y_train = conv_data_pd_salt(train_data_salt, train_data_salt, fp_salt, en_salt, sc_salt,numeric_features_salt )
x_train=x_train_pd.values
model_salt = XGBRegressor(max_delta_step=params['max_delta_step'], learning_rate = params['learning_rate'],
                    max_depth = params['max_depth'], min_child_weight= params['min_child_weight'],
                    subsample=params['subsample'],reg_alpha=params['reg_alpha'],gamma= params['gamma'],
                    reg_lambda= params['reg_lambda'],n_estimators=params['n_estimators'])

In [ ]:
###Checking the model
model_salt.load_model('rej_model.model')
x_train_pd, y_train = conv_data_pd_salt(train_data_salt, train_data_salt, fp_salt, en_salt, sc_salt, numeric_features_salt)
x_train=x_train_pd.values
r2_score(y_train, model_salt.predict(x_train))

In [ ]:
col = list(train_data_salt.columns)
col.append('Permeability')

In [ ]:
###limited momonmer
mom_l = pd.read_excel('top_5.xlsx', sheet_name='Sheet2')
mom_l.head(20)

In [ ]:
mom_l.loc[14, 'Monomer A1 type']='carbon'
mom_l.loc[14, 'a-smile']='C'
mom_l.loc[14, 'a-mw']=0

In [ ]:

X1T = mom_l[[' Additive X1 type in aqueous phase']].copy()
X1T.drop_duplicates(keep='first', inplace=True)
X1T.reset_index(drop=True, inplace=True)
X1T.head()


In [ ]:

sub = mom_l[['Substrate membrane type', 'substrate membrane pore size']].copy()
sub.drop_duplicates(keep='first', inplace=True)
sub.reset_index(drop=True, inplace=True)
sub.drop(index = 3, inplace=True)
sub.head()


In [ ]:
###optimization
space = {
    'a-smile': hp.randint('a-smile', len(mom_l)),
    'b-smile': hp.randint('b-smile', len(mom_l)),
    #'Monomer B type': hp.randint('Monomer B type', len(MB)),
    #'Organic solvent type': hp.randint('Organic solvent type', len(OB)),
    ' Additive X1 type in aqueous phase': hp.randint(' Additive X1 type in aqueous phase', len(X1T)),
    #' Additive X2 type in aqueous phase': hp.randint(' Additive X2 type in aqueous phase', len(X2T)),
    #' Additive Y type in organic  phase':hp.randint(' Additive Y type in aqueous phase', len(YT)),
    #'Nanomaterials type in aqueous phase':hp.randint('Nanomaterials type in aqueous phase', len(NTA)),
    #'Nanomaterials type in organic phase':hp.randint('Nanomaterials type in organic phase', len(NTO)),
   'Substrate membrane type':hp.randint('Substrate membrane type', len(sub)),
    #'nanomaterials morphology':hp.randint('nanomaterials morphology', len(NS)),
    'A2/A1 ratio': hp.uniform('A1/A2 ratio', 0, 8),
    'Monomer A concentration': hp.uniform('Monomer A concentration', 0.01, 3),
    'Monomer B concentration': hp.uniform('Monomer B concentration', 0.01, 1),
    'Additive X1 concentration': hp.uniform('Additive X1 concentration', 0.01, 0.6),
    #'Additive X2 concentration': hp.choice('Additive X2 concentration',[hp.uniform('Additive X2 concentration_a', 0, 10), np.nan]),
    #'Aqueous phase pH':hp.choice('Aqueous phase pH', [hp.uniform('Aqueous phase pH_a', 7, 12), np.nan]),
    #'Additive Y concentration':hp.choice('Additive Y concentration', [hp.uniform('Additive Y concentration_a', 0, 2), np.nan]),
    'Nanomaterials loading in aqueous phase': hp.uniform('Nanomaterials loading in aqueous phase_a', 0.001, 0.1),
    #'Nanomaterials loading in organnic phase':hp.uniform('Nanomaterials loading in organnic phase', 0, 4),
    'Polymerization time':hp.quniform('Polymerization time_a', 5, 600, 1),
    'Post-treatment time':hp.quniform('Post-treatment time_a', 1, 30,1),
    'post-treatment T': hp.quniform('post-treatment T_a', 22, 65,1), 
    'TMP':hp.quniform('TMP', 2, 10,1)
        
}

In [ ]:
##NaCl
def get_x_l(params):
    hh = pd.DataFrame(columns=col)
    hh.loc[0, 'Monomer A1 type']=mom_l.loc[params['a-smile'],'Monomer A1 type']
    hh.loc[0,'a-smile']=mom_l.loc[params['a-smile'],'a-smile']
    hh.loc[0,'a-mw']=mom_l.loc[params['a-smile'],'a-mw']
    hh.loc[0, 'Monomer A2 type']=mom_l.loc[params['b-smile'],'Monomer A1 type']
    hh.loc[0, 'b-smile']=mom_l.loc[params['b-smile'],'a-smile']
    hh.loc[0, 'b-mw']=mom_l.loc[params['b-smile'],'a-mw']
    hh.loc[0, 'A2/A1 ratio']=params['A2/A1 ratio']
    hh.loc[0, 'Monomer A concentration']=params['Monomer A concentration']
    hh.loc[0,'Monomer B concentration']=params['Monomer B concentration']
    hh.loc[0, 'Additive X1 concentration']=params['Additive X1 concentration']
    hh.loc[0, 'Additive X2 concentration']=0
    hh.loc[0, 'Aqueous phase pH']=np.nan
    hh.loc[0,'Additive Y concentration']=0
    hh.loc[0, 'Nanomaterials loading in aqueous phase']=params['Nanomaterials loading in aqueous phase']
    hh.loc[0, 'Nanomaterials loading in organnic phase']=0
    hh.loc[0, 'Polymerization time']=params['Polymerization time']
    hh.loc[0, 'Post-treatment time']=params['Post-treatment time']
    hh.loc[0,'post-treatment T']=params['post-treatment T']
    hh.loc[0,'TMP']=params['TMP']


    hh.loc[0, 'Monomer B type']='trimesoyl chloride (265.5)'
    hh.loc[0, 'Substrate membrane type']=sub.loc[params['Substrate membrane type'], 'Substrate membrane type']
    hh.loc[0, 'nanomaterials morphology']= np.nan
    hh.loc[0,'Organic solvent type']= 'Hexane'
    hh.loc[0, ' Additive X1 type in aqueous phase']= X1T.loc[params[' Additive X1 type in aqueous phase'], ' Additive X1 type in aqueous phase']
    hh.loc[0, ' Additive X2 type in aqueous phase']= np.nan
    hh.loc[0, ' Additive Y type in organic  phase']= np.nan
    hh.loc[0, 'Nanomaterials type in aqueous phase']= 'CNC'
    hh.loc[0, 'Nanomaterials type in organic phase']= np.nan
    
    
    
    hh.loc[0, 'Cation valence']=data_NaSO4['Na+ valence'][0]
    hh.loc[0,'Cation ionic radius' ]=data_NaSO4['Na+ ionic radius'][0]
    hh.loc[0, 'Cation hydrated radius']=data_NaSO4['Na+ hydrated radius'][0]
    hh.loc[0, 'Cation stokes radius']=data_NaSO4['Na+ stokes radius'][0]
    hh.loc[0, 'Cation hydration free energy']=data_NaSO4['Na+ hydration free energy'][0]
    hh.loc[0, 'Anion valence']=data_NaSO4['SO42- valence'][0]
    hh.loc[0, 'Anion ionic radius']=data_NaSO4['SO42-  ionic radius'][0]
    hh.loc[0, 'Anion hydrated radius']=data_NaSO4['SO42-  hydrated radius'][0]
    hh.loc[0, 'Anion  stokes radius']=data_NaSO4['SO42-  stokes radius'][0]
    hh.loc[0, 'Anion  hydration free energy']=data_NaSO4['SO42-  hydration free energy'][0]
    hh.loc[0, 'salt concentration']=1000
    hh.loc[0, 'substrate membrane pore size']=sub.loc[params['Substrate membrane type'],'substrate membrane pore size']
    hh.loc[0, 'salt rejection']=np.nan
    hh.loc[0, 'Permeability']=np.nan
    
    return hh

In [ ]:
from hyperopt.pyll.stochastic import sample
hh = sample(space)

In [ ]:
d = get_x_l(hh)

In [ ]:
def fit(params):
    hh= get_x_l(params)
    if hh['a-smile'][0]==hh['b-smile'][0]:
        hh.loc[0, 'b-smile']='C'
        hh.loc[0, 'b-mw']=0
    for i in hh[hh[' Additive X1 type in aqueous phase'].isnull()==True].index:
        hh.loc[i, 'Additive X1 concentration']=0
    if hh['b-smile'][0]=='C':
        hh.loc[0,'A2/A1 ratio']=0
    x_pd_salt,_ = conv_data_pd_salt(hh[train_data_salt.columns], train_data_salt, fp_salt, en_salt,sc_salt, numeric_features_salt)
    x_salt=x_pd_salt.values
    x_pd_per,_ = conv_data_pd_per(hh[train_data_per.columns],train_data_per, fp_per, en_per,sc_per, numeric_features_per )
    x_per=x_pd_per.values
    pred = model_salt.predict(x_salt)[0]
    per_pred = model_per.predict(x_per)[0]
    return abs((pred/((100-pred)*params['TMP']))-60) + abs(per_pred -19), pred/((100-pred)*params['TMP']), pred, per_pred

def objective(params):
    global ITERATION
    ITERATION +=1
    loss,y, salt, per = fit(params)
    off_connection = open(out_file, 'a')
    writer = csv.writer(off_connection)
    writer.writerow([loss,y, salt, per, params, ITERATION])
    return {'loss':loss,'Y':y,'salt':salt, 'per':per, 'params': params, 'iteration':ITERATION, 'status':STATUS_OK}

import csv
out_file ='salt_opt_SO4_19_60_withnm.csv'
off_connection =open(out_file, 'w')
writer = csv.writer(off_connection)
writer.writerow(['loss','Y','salt', 'per', 'params', 'iteration'])
off_connection.close()

tpe_algo = tpe.suggest
bayes_trial = Trials()

In [ ]:
##%%capture
global ITERATION
ITERATION =0
best = fmin(fn = objective, space =space, algo = tpe_algo, trials = bayes_trial, max_evals=24000, rstate= np.random.RandomState(50)) 

In [ ]:
##NaCL
result = pd.read_csv('salt_opt_SO4_19_60_withnm.csv')
result.sort_values('Y', ascending= False, inplace = True)
result.reset_index(drop = True, inplace =True)
result.head(20)

In [ ]:
##NaCL
result = pd.read_csv('salt_opt_SO4_19_60.csv')
result.sort_values('per', ascending= False, inplace = True)
result.reset_index(drop = True, inplace =True)
result.head(10)

In [ ]:
result['params'][3]

In [ ]:
tt=result.loc[0:50]
tt.to_csv('NaSO4_19_60_withnm.csv', index=False)

In [ ]:
hh = pd.DataFrame(columns=col)

In [ ]:
import ast
for i in range(20):
    params = ast.literal_eval(result['params'][i])
    cc= get_x_l(params)
    if cc['a-smile'][0]==cc['b-smile'][0]:
        cc.loc[0, 'b-smile']='C'
        cc.loc[0, 'b-mw']=0
    for i in cc[cc[' Additive X1 type in aqueous phase'].isnull()==True].index:
        cc.loc[i, 'Additive X1 concentration']=0
    if cc['b-smile'][0]=='C':
        cc.loc[0,'A2/A1 ratio']=0
    hh = pd.concat([hh, cc], ignore_index=True)


In [ ]:
hh.head()

In [ ]:
hh.to_excel('NaSO4_19_60_condition_withnm.xlsx', index=False)